In [119]:
import time
import pandas as pd
import numpy as np
import warnings
import feather
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)

There are many insignificant features to our work, like codes, PK, FK and out of context. 
Based on prior works and experience of especialists, will be taken off nonsense features. 
A study was carried out on the evolution and correspondence of these resources in the ENEM and Census
data for the period 2009-2018. See (featuresOrganisaton.xls)

In [194]:
teacher = ['ID_DOCENTE', 'CO_ENTIDADE','IN_ESPECIALIZACAO','IN_MESTRADO','IN_DOUTORADO']

school = ['NU_ANO_CENSO','CO_ENTIDADE', 'CO_MUNICIPIO','CO_UF','TP_DEPENDENCIA','IN_AGUA_INEXISTENTE',
                  'IN_ESGOTO_INEXISTENTE','IN_ENERGIA_INEXISTENTE',
                   'IN_LABORATORIO_INFORMATICA','IN_LABORATORIO_CIENCIAS','IN_SALA_ATENDIMENTO_ESPECIAL',
                   'IN_BIBLIOTECA','IN_SALA_LEITURA','IN_BANHEIRO_FORA_PREDIO',
                   'IN_BANHEIRO_PNE','IN_DEPENDENCIAS_PNE','QT_SALAS_UTILIZADAS','QT_EQUIP_TV',
                   'QT_EQUIP_DVD','QT_EQUIP_COPIADORA','QT_EQUIP_IMPRESSORA','QT_COMP_ADMINISTRATIVO',
                   'QT_COMP_ALUNO','IN_INTERNET','IN_BANDA_LARGA','QT_FUNCIONARIOS','IN_ALIMENTACAO']


enem = ['NU_ANO','CO_ESCOLA','TP_ENSINO','NU_IDADE','TP_SEXO','TP_COR_RACA','NU_NOTA_CN',
        'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO','Q001','Q002','Q005','Q006'
]

In [122]:
#Students of ENEM
E = pd.read_csv('~/data/enem/2018/DADOS/MICRODADOS_ENEM_2018.csv', sep=';', encoding="iso-8859-2", usecols = enem)

In [124]:
#School census data
CE =pd.read_csv("~/data/censo/2018/DADOS/ESCOLAS.CSV", sep='|', encoding="iso-8859-2", usecols = school)

In [125]:
#Teacher census data by Braazilian mesoregion
CD_CO = pd.read_csv("~/data/censo/2018/DADOS/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_NE = pd.read_csv("~/data/censo/2018/DADOS/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_N =  pd.read_csv("~/data/censo/2018/DADOS/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_SE = pd.read_csv("~/data/censo/2018/DADOS/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_S =  pd.read_csv("~/data/censo/2018/DADOS/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)

#All teacher census data
CT = pd.concat([CD_CO, CD_NE, CD_N, CD_SE, CD_S])

In [231]:
ENEM = E.copy()
C_SCHOOL = CE.copy()
C_TEACHER = CT.copy()

New Names and Checking dtypes of COD's

In [232]:
#ordering
ENEM = ENEM[enem]

newNames = [
'CO_ANO',
'CO_ESCOLA',
'TP_ENSINO',
'NU_IDADE',
'TP_SEXO',
'TP_COR_RACA',
'NU_NOTA_CN',
'NU_NOTA_CH',
'NU_NOTA_LC',
'NU_NOTA_MT',
'NU_NOTA_REDACAO',
'EDU_PAI',
'EDU_MAE',
'QT_PESSOAS_CASA',
'RENDA_MENSAL'
]

#Rename
ENEM.columns = newNames

print('checking year',ENEM.CO_ANO.unique())
print('checking key',ENEM.CO_ESCOLA.dtypes)

checking year [2018]
checking key float64


In [233]:
C_SCHOOL = C_SCHOOL[school]

newNames = ['ANO_CENSO',
'CO_ESCOLA',
 'CO_MUNICIPIO',
 'CO_UF',
 'CO_DEPENDENCIA_ADM',
 'IN_AGUA_INEXISTENTE',
 'IN_ESGOTO_INEXISTENTE',
 'IN_ENERGIA_INEXISTENTE',
 'IN_LABORATORIO_INFORMATICA',
 'IN_LABORATORIO_CIENCIAS',
 'IN_SALA_ATENDIMENTO_ESPECIAL',
 'IN_BIBLIOTECA',
 'IN_SALA_LEITURA',
 'IN_BANHEIRO_FORA_PREDIO',
 'IN_BANHEIRO_PNE',
 'IN_DEPENDENCIAS_PNE',
 'QT_SALAS_UTILIZADAS',
 'QT_EQUIP_TV',
 'QT_EQUIP_DVD',
 'QT_EQUIP_COPIADORA',
 'QT_EQUIP_IMPRESSORA',
 'QT_COM_ADMINISTRATIVO',
 'QT_COMP_ALUNO',
 'IN_INTERNET',
 'IN_BANDA_LARGA',
 'QT_FUNCIONARIOS',
 'IN_ALIMENTACAO'
]

#Rename 
C_SCHOOL.columns =  newNames

print('Checking year of censo',C_SCHOOL.ANO_CENSO.unique())
C_SCHOOL.drop('ANO_CENSO', axis=1, inplace=True)
print('checking key',C_SCHOOL.CO_ESCOLA.dtypes)

Checking year of censo [2018]
checking key int64


In [234]:
C_TEACHER = C_TEACHER[teacher]

# New default column names for all Datasets
newNames = [
'CO_PROFESSOR',
'CO_ESCOLA',  
'IN_ESPECIALIZACAO',
'IN_MESTRADO',
'IN_DOUTORADO'
]
#It's necessary drop duplicates whose are teachers in the same school but in different classes. 
#This way, the teacher appear just one time in each school. 
C_TEACHER.columns = newNames
C_TEACHER.drop_duplicates(subset=['CO_PROFESSOR', 'CO_ESCOLA'], inplace=True)
C_TEACHER.drop('CO_PROFESSOR', axis=1, inplace = True)

print('checking key',C_TEACHER.CO_ESCOLA.dtypes)

checking key int64


### Minimum Scope Definition

In [235]:
ENEM.dropna(inplace = True, axis=0, subset = ['CO_ESCOLA'])

In [236]:
#should care about the nulls into scope filter features?
print(ENEM['TP_ENSINO'].value_counts())
print(ENEM['TP_ENSINO'].isnull().sum())
#NO, IT'S LOWER

1.0    1308998
3.0     134477
2.0       5171
Name: TP_ENSINO, dtype: int64
175


In [237]:
print('drop out',((ENEM.TP_ENSINO!= 1)|(ENEM.TP_ENSINO!= 3)).sum(), 'who not attend regular and prof approachs schools')
ENEM = ENEM.loc[((ENEM.TP_ENSINO== 1)|(ENEM.TP_ENSINO== 3))]
ENEM.drop('TP_ENSINO', inplace = True, axis =1)

drop out 1448821 who not attend regular and prof approachs schools


In [238]:
fields = (['NU_NOTA_CN', 'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT', 'NU_NOTA_REDACAO'])
for i in fields:
    print ('null:', i, ENEM[i].isnull().sum())
    print ('zero:', i, (ENEM[i]== 0).sum())

null: NU_NOTA_CN 313568
zero: NU_NOTA_CN 161
null: NU_NOTA_CH 254947
zero: NU_NOTA_CH 2224
null: NU_NOTA_LC 254947
zero: NU_NOTA_LC 1000
null: NU_NOTA_MT 313568
zero: NU_NOTA_MT 219
null: NU_NOTA_REDACAO 254947
zero: NU_NOTA_REDACAO 37380


In [239]:
ENEM.dropna(inplace = True, axis=0, subset=fields)
ENEM = ENEM.loc[~(ENEM[fields] == 0).any(axis=1)]

In [240]:
#ONLY STUDENTS AT  17 -19 YEAR OLD. WE WANT TO GET ONLY REGULAR STUDENTS THAT ARE IN LAST YEAR OF SECONDARY SCHOOL. 
#THE MICRODADOS HAVE A FIELD TO CONTROL IT. BUT IT IS NOT CONFIDENCE,DUE A TURN OUT TEAACHERS THAT EXTEND ACADEMIC YEAR
print('null', ENEM.NU_IDADE.isnull().sum())
#firts, fill NA with the mean of student school
ENEM.NU_IDADE = ENEM.groupby(ENEM['CO_ESCOLA'])['NU_IDADE'].apply(lambda x: x.fillna(round(x.mean())))
print('taking off',((ENEM.NU_IDADE <17)|(ENEM.NU_IDADE>19)).sum())
ENEM = ENEM.loc[(ENEM.NU_IDADE >16) &  (ENEM.NU_IDADE<20)]
ENEM.NU_IDADE.value_counts()

null 35
taking off 102778


17.0    514758
18.0    387216
19.0     93345
Name: NU_IDADE, dtype: int64

In [241]:
ENEM[fields].describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,995319.000000,995319.000000,995319.000000,995319.000000,995319.000000
mean,496.770612,570.675155,530.106678,542.462418,541.396698
std,75.335773,77.234482,71.069319,103.988189,174.967770
min,362.500000,318.700000,318.800000,360.000000,40.000000
25%,437.000000,512.400000,480.300000,461.500000,380.000000
50%,486.300000,583.500000,535.400000,523.700000,540.000000
75%,547.900000,629.600000,582.500000,609.900000,660.000000
max,869.600000,850.400000,816.900000,996.100000,1000.000000


### Transform features to default categories

In [242]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL']
ENEM[fields].isnull().sum()

TP_SEXO            0
TP_COR_RACA        0
EDU_PAI            0
EDU_MAE            0
QT_PESSOAS_CASA    0
RENDA_MENSAL       0
dtype: int64

In [243]:
for i in fields:
    print(i,ENEM[i].value_counts().index)

TP_SEXO Index(['F', 'M'], dtype='object')
TP_COR_RACA Int64Index([3, 1, 2, 4, 0, 5], dtype='int64')
EDU_PAI Index(['E', 'B', 'C', 'D', 'F', 'H', 'G', 'A'], dtype='object')
EDU_MAE Index(['E', 'D', 'C', 'F', 'B', 'G', 'H', 'A'], dtype='object')
QT_PESSOAS_CASA Int64Index([4, 3, 5, 2, 6, 7, 8, 9, 10, 1, 11, 12, 13, 20, 14, 15, 16, 17, 18,
            19],
           dtype='int64')
RENDA_MENSAL Index(['C', 'B', 'E', 'D', 'G', 'F', 'H', 'I', 'A', 'J', 'N', 'K', 'Q', 'L',
       'O', 'P', 'M'],
      dtype='object')


In [244]:
#Order relation, is better for numeric categories.
categorical = ['TP_SEXO', 'EDU_PAI', 'EDU_MAE', 'RENDA_MENSAL']
for i in categorical:
    codes, uniques = pd.factorize(ENEM[i], sort=True)
    ENEM[i] = codes

In [245]:
ENEM['EDU_PAI_TEMP'] = ENEM['EDU_PAI']
ENEM['EDU_MAE_TEMP'] = ENEM['EDU_MAE']
ENEM['QT_PESSOAS_CASA_TEMP'] = ENEM['QT_PESSOAS_CASA']
ENEM['RENDA_MENSAL_TEMP'] = ENEM['RENDA_MENSAL']
ENEM['TP_COR_RACA_TEMP'] = ENEM['TP_COR_RACA']

In [246]:
# Paring with default categories

ENEM.loc[ENEM['EDU_PAI']==0, 'EDU_PAI_TEMP']=0
ENEM.loc[ENEM['EDU_PAI']==1, 'EDU_PAI_TEMP']=0
ENEM.loc[ENEM['EDU_PAI']==2, 'EDU_PAI_TEMP']=1
ENEM.loc[ENEM['EDU_PAI']==3, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==4, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==5, 'EDU_PAI_TEMP']=4
ENEM.loc[ENEM['EDU_PAI']==6, 'EDU_PAI_TEMP']=5
ENEM.loc[ENEM['EDU_PAI']==7, 'EDU_PAI_TEMP']=0

ENEM.loc[ENEM['EDU_MAE']==0, 'EDU_MAE_TEMP']=0
ENEM.loc[ENEM['EDU_MAE']==1, 'EDU_MAE_TEMP']=0
ENEM.loc[ENEM['EDU_MAE']==2, 'EDU_MAE_TEMP']=1
ENEM.loc[ENEM['EDU_MAE']==3, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==4, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==5, 'EDU_MAE_TEMP']=4
ENEM.loc[ENEM['EDU_MAE']==6, 'EDU_MAE_TEMP']=5
ENEM.loc[ENEM['EDU_MAE']==7, 'EDU_MAE_TEMP']=0



ENEM.loc[ENEM['QT_PESSOAS_CASA']==1, 'QT_PESSOAS_CASA_TEMP']=0
ENEM.loc[ENEM['QT_PESSOAS_CASA']==2, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==3, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==4, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==5, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==6, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==7, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']>7, 'QT_PESSOAS_CASA_TEMP']=3

ENEM.loc[ENEM['RENDA_MENSAL']==0, 'RENDA_MENSAL_TEMP']=0
ENEM.loc[ENEM['RENDA_MENSAL']==1, 'RENDA_MENSAL_TEMP']=1
ENEM.loc[ENEM['RENDA_MENSAL']==2, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[ENEM['RENDA_MENSAL']==3, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[(ENEM['RENDA_MENSAL']>3) & (ENEM['RENDA_MENSAL']<8), 'RENDA_MENSAL_TEMP']=3
ENEM.loc[(ENEM['RENDA_MENSAL']>7) & (ENEM['RENDA_MENSAL']<13), 'RENDA_MENSAL_TEMP']=4
ENEM.loc[ENEM['RENDA_MENSAL']>12, 'RENDA_MENSAL_TEMP']=5

In [247]:
#back to original columns
ENEM['EDU_PAI'] = ENEM['EDU_PAI_TEMP']
ENEM['EDU_MAE'] = ENEM['EDU_MAE_TEMP']
ENEM['QT_PESSOAS_CASA'] = ENEM['QT_PESSOAS_CASA_TEMP']
ENEM['RENDA_MENSAL'] = ENEM['RENDA_MENSAL_TEMP']

In [248]:
#drop temps

ENEM.drop(['EDU_PAI_TEMP', 'EDU_MAE_TEMP', 'QT_PESSOAS_CASA_TEMP','RENDA_MENSAL_TEMP'], axis=1, inplace=True)

In [249]:
print('Total Students at ENEM:', ENEM.shape[0], 'of', ENEM.drop_duplicates('CO_ESCOLA').shape[0], 'schools')
print('Total Schools at Census:', C_SCHOOL.shape[0])

#######################################
print('########### Only about Schools at ENEM:')

schoolsEnem = ENEM['CO_ESCOLA'].dropna().unique()
schoolsCenso = C_SCHOOL['CO_ESCOLA'].dropna().unique()
schoolsTeacher = C_TEACHER['CO_ESCOLA'].dropna().unique()
schoolsEnem2 = ENEM['CO_ESCOLA'].dropna()

enem_censo = np.setdiff1d(schoolsEnem ,schoolsCenso)
print('Schools at ENEM out of School Census', sum(np.isin(schoolsEnem, schoolsCenso, invert=True)))

print('Schools at ENEM out of Teacher Census', sum(np.isin(schoolsEnem, schoolsTeacher, invert=True)))
print('Schools at ENEM and School Census out of Teacher Census', sum(np.isin(enem_censo, schoolsTeacher)))

print('Students Wasted*****', sum(np.isin(schoolsEnem2, schoolsCenso, invert=True)))


Total Students at ENEM: 995319 of 29637 schools
Total Schools at Census: 286014
########### Only about Schools at ENEM:
Schools at ENEM out of School Census 82
Schools at ENEM out of Teacher Census 753
Schools at ENEM and School Census out of Teacher Census 0
Students Wasted***** 646


In [250]:
#Final Media, Target
ENEM['NU_NOTA_GERAL'] = (ENEM.NU_NOTA_CN + ENEM.NU_NOTA_CH + ENEM.NU_NOTA_LC + ENEM.NU_NOTA_MT + ENEM.NU_NOTA_REDACAO)/5
ENEM['TARGET'] = pd.qcut (ENEM.NU_NOTA_GERAL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 
print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

75.00509886780017 % lowers quartis


In [251]:
C_TEACHER.isnull().sum()

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


CO_ESCOLA                 0
IN_ESPECIALIZACAO    754634
IN_MESTRADO          754634
IN_DOUTORADO         754634
dtype: int64

In [252]:
#lets fill it with lower level

C_TEACHER['IN_ESPECIALIZACAO'].fillna(0, inplace = True)
C_TEACHER['IN_MESTRADO'].fillna(0, inplace = True)
C_TEACHER['IN_DOUTORADO'].fillna(0, inplace = True)

In [253]:
C_TEACHER['TITULACAO'] = 0


C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 1

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_DOUTORADO ==1.0), 'TITULACAO'] = 3

SUM= C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform(sum)
COUNT =  C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform('count')
C_TEACHER['TITULACAO'] = (SUM)/(COUNT*3)   

#transformin to school grain
C_TEACHER.drop_duplicates('CO_ESCOLA', inplace = True)

## STUDENT GRAIN

In [230]:
# teacher to Schoool Grain
print('C_SCHOOL',C_SCHOOL.shape)
print('C_TEACHER',C_TEACHER.shape)
CENSO19 = pd.merge(C_SCHOOL, C_TEACHER, on = 'CO_ESCOLA', how = 'inner')
print('C_SCHOOL + C_TEACHER',CENSO19.shape)

#Now, all to Student Grain
print('ENEM',ENEM.shape)
ENEM_CENSO_19 = pd.merge(ENEM, CENSO19, on = 'CO_ESCOLA', how = 'inner')
print('FINAL',ENEM_CENSO_19.shape)

ENEM_CENSO_19.to_csv('STUDENT.csv')


C_SCHOOL (286014, 26)
C_TEACHER (183746, 5)
C_SCHOOL + C_TEACHER (183746, 30)
ENEM (29637, 16)
FINAL (28884, 45)


In [225]:
ENEM_CENSO_19.head()

,CO_ANO,CO_ESCOLA,NU_IDADE,TP_SEXO,TP_COR_RACA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,EDU_PAI,EDU_MAE,QT_PESSOAS_CASA,RENDA_MENSAL,NU_NOTA_GERAL,TARGET,CO_MUNICIPIO,CO_UF,CO_DEPENDENCIA_ADM,IN_AGUA_INEXISTENTE,IN_ESGOTO_INEXISTENTE,IN_ENERGIA_INEXISTENTE,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,QT_SALAS_UTILIZADAS,QT_EQUIP_TV,QT_EQUIP_DVD,QT_EQUIP_COPIADORA,QT_EQUIP_IMPRESSORA,QT_COM_ADMINISTRATIVO,QT_COMP_ALUNO,IN_INTERNET,IN_BANDA_LARGA,QT_FUNCIONARIOS,IN_ALIMENTACAO,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,TITULACAO
0,2018,23077140.0,19.0,1,3,411.8,429.8,445.2,459.0,340.0,2.0,2.0,2.0,2.0,417.16,0,2304400,23,2,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,0.0,1.0,1.0,7.0,10.0,1.0,1.0,122.0,1.0,0.0,0.0,0.0,0.098958
1,2018,23077140.0,18.0,0,3,457.3,564.2,474.6,462.9,300.0,2.0,2.0,2.0,2.0,451.80,0,2304400,23,2,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,0.0,1.0,1.0,7.0,10.0,1.0,1.0,122.0,1.0,0.0,0.0,0.0,0.098958
2,2018,23077140.0,18.0,0,3,410.5,474.8,515.6,490.7,540.0,2.0,2.0,2.0,2.0,486.32,0,2304400,23,2,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,0.0,1.0,1.0,7.0,10.0,1.0,1.0,122.0,1.0,0.0,0.0,0.0,0.098958
3,2018,23077140.0,17.0,0,3,506.9,590.0,493.2,438.6,300.0,2.0,2.0,2.0,2.0,465.74,0,2304400,23,2,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,0.0,1.0,1.0,7.0,10.0,1.0,1.0,122.0,1.0,0.0,0.0,0.0,0.098958
4,2018,23077140.0,19.0,1,3,446.1,566.0,343.7,468.2,260.0,2.0,2.0,2.0,2.0,416.80,0,2304400,23,2,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,0.0,1.0,1.0,7.0,10.0,1.0,1.0,122.0,1.0,0.0,0.0,0.0,0.098958


## SCHOOL GRAIN

In [226]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL', 'NU_IDADE', 'NU_NOTA_CN',
             'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO','NU_NOTA_GERAL' ]

for i in fields:   
    
            ENEM[i] = ENEM.groupby('CO_ESCOLA')[i].transform('mean')
            

In [227]:
# SCHOOL GRAIN AND SHOW THE NEW NOTA DISTRIBUTION
ENEM.drop_duplicates('CO_ESCOLA', inplace=True)
print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

74.76802645341971 % lowers quartis


In [228]:
#recalculating target for new  Nota Geral distribution 
ENEM['TARGET'] = pd.qcut (ENEM.NU_NOTA_GERAL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 
print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

75.00084354016938 % lowers quartis


In [229]:
print('censo before',CENSO19.shape)
print('enem_school before', ENEM.shape)
CENSO_ENEM_19 = pd.merge(CENSO19, ENEM, on='CO_ESCOLA', how='inner')
print(CENSO_ENEM_19.shape)

CENSO_ENEM_19.to_csv('SCHOOL.csv', index = False)


censo before (183746, 30)
enem_school before (29637, 16)
(28884, 45)


In [255]:
CENSO_ENEM_19.isnull().sum()

CO_ESCOLA                          0
CO_MUNICIPIO                       0
CO_UF                              0
CO_DEPENDENCIA_ADM                 0
IN_AGUA_INEXISTENTE                0
IN_ESGOTO_INEXISTENTE              0
IN_ENERGIA_INEXISTENTE             0
IN_LABORATORIO_INFORMATICA         0
IN_LABORATORIO_CIENCIAS            0
IN_SALA_ATENDIMENTO_ESPECIAL       0
IN_BIBLIOTECA                      0
IN_SALA_LEITURA                    0
IN_BANHEIRO_FORA_PREDIO            0
IN_BANHEIRO_PNE                    0
IN_DEPENDENCIAS_PNE                0
QT_SALAS_UTILIZADAS                0
QT_EQUIP_TV                        0
QT_EQUIP_DVD                       0
QT_EQUIP_COPIADORA                 0
QT_EQUIP_IMPRESSORA                0
QT_COM_ADMINISTRATIVO              0
QT_COMP_ALUNO                      0
IN_INTERNET                        0
IN_BANDA_LARGA                  1158
QT_FUNCIONARIOS                    0
IN_ALIMENTACAO                     0
IN_ESPECIALIZACAO                  0
I

In [256]:
CENSO_ENEM_19.head()

,CO_ESCOLA,CO_MUNICIPIO,CO_UF,CO_DEPENDENCIA_ADM,IN_AGUA_INEXISTENTE,IN_ESGOTO_INEXISTENTE,IN_ENERGIA_INEXISTENTE,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,QT_SALAS_UTILIZADAS,QT_EQUIP_TV,QT_EQUIP_DVD,QT_EQUIP_COPIADORA,QT_EQUIP_IMPRESSORA,QT_COM_ADMINISTRATIVO,QT_COMP_ALUNO,IN_INTERNET,IN_BANDA_LARGA,QT_FUNCIONARIOS,IN_ALIMENTACAO,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,TITULACAO,CO_ANO,NU_IDADE,TP_SEXO,TP_COR_RACA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,EDU_PAI,EDU_MAE,QT_PESSOAS_CASA,RENDA_MENSAL,NU_NOTA_GERAL,TARGET
0,11022981,1100320,11,2,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,21.0,8.0,0.0,1.0,9.0,15.0,10.0,1.0,1.0,107.0,1.0,1.0,0.0,0.0,0.306667,2018,17.649123,0.403509,2.342105,465.984211,528.496491,490.143860,495.773684,466.842105,1.166667,1.807018,1.666667,2.008772,489.448070,0
1,11023023,1100346,11,2,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,11.0,4.0,3.0,0.0,10.0,13.0,18.0,1.0,1.0,45.0,1.0,1.0,0.0,0.0,0.208333,2018,18.500000,0.500000,2.500000,427.400000,496.550000,468.550000,432.100000,300.000000,0.000000,0.500000,2.000000,1.500000,424.920000,0
2,11049430,1100205,11,2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,20.0,4.0,0.0,2.0,4.0,7.0,26.0,1.0,1.0,43.0,1.0,0.0,0.0,0.0,0.103448,2018,18.041667,0.375000,2.541667,468.204167,524.433333,494.212500,502.066667,492.500000,1.250000,1.833333,1.750000,1.833333,496.283333,0
3,12000213,1200203,12,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,15.0,1.0,5.0,2.0,7.0,6.0,0.0,1.0,1.0,48.0,1.0,1.0,0.0,0.0,0.065041,2018,17.627907,0.418605,2.744186,439.116279,507.774419,490.132558,474.095349,439.534884,1.186047,1.651163,2.046512,1.325581,470.130698,0
4,12003760,1200427,12,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,35.0,1.0,2.0,0.0,2.0,1.0,0.0,1.0,0.0,58.0,1.0,0.0,0.0,0.0,0.000000,2018,17.500000,0.000000,3.000000,417.400000,490.750000,423.900000,458.900000,520.000000,0.000000,0.000000,2.000000,0.500000,462.190000,0
